# Mamba-MoE 509M - Production Training (Pre-Downloaded Data)

**Optimized for Kaggle TPU v5e-8**

- Uses existing FineWeb-Edu dataset (no re-download!)
- Checkpoints every 10K steps (saves storage)
- ~50K total steps = 10-12 hours

## Step 1: Setup & Install Dependencies

In [ ]:
# Clone repo (or use existing)
%cd /kaggle/working
!git clone https://github.com/rgprince/mamba-moe-300m.git 2>/dev/null || echo "Repo exists"
%cd mamba-moe-300m

# Pull latest fixes
!git pull origin main

# Install dependencies
!pip install -q transformers

print("\n✅ Setup complete!")

## Step 2: Load Pre-Downloaded Dataset

**Using your existing FineWeb-Edu .npy files!**

In [ ]:
import numpy as np
from pathlib import Path
from tqdm import tqdm

print("="*70)
print("LOADING PRE-DOWNLOADED DATASET")
print("="*70)

# Path to your dataset
DATASET_PATH = Path("/kaggle/input/fineweb-edu-10bt-for-gpt2/train")

# Find all .npy files
npy_files = sorted(DATASET_PATH.glob("fineweb_edu_*.npy"))
print(f"\nFound {len(npy_files)} dataset files")

# Load all data into memory (or use lazy loading if too big)
all_token_ids = []

for npy_file in tqdm(npy_files, desc="Loading files"):
    data = np.load(npy_file)
    all_token_ids.extend(data.tolist())

print(f"\n✓ Loaded {len(all_token_ids):,} tokens")
print(f"  Dataset size: {len(all_token_ids) * 2 / 1e9:.2f}GB (int16)")
print(f"  Estimated training tokens: {len(all_token_ids) / 1e6:.1f}M")

# Convert to array for efficient slicing
token_ids = np.array(all_token_ids, dtype=np.int32)
del all_token_ids  # Free memory

print(f"\n📊 Dataset Statistics:")
print(f"  Min token ID: {token_ids.min()}")
print(f"  Max token ID: {token_ids.max()}")
print(f"  Mean: {token_ids.mean():.1f}")

print(f"\n✅ Dataset loaded!")

## Step 3: Load GPT-2 Tokenizer

In [ ]:
from transformers import GPT2TokenizerFast

print("="*70)
print("LOADING GPT-2 TOKENIZER")
print("="*70)

tokenizer_hf = GPT2TokenizerFast.from_pretrained("gpt2")

# Wrapper for compatibility
class TokenizerWrapper:
    def __init__(self, hf_tokenizer):
        self.tokenizer = hf_tokenizer
        self.vocab_size = len(hf_tokenizer)
        self.bos_id = hf_tokenizer.bos_token_id or 50256
        self.eos_id = hf_tokenizer.eos_token_id or 50256
        self.pad_id = hf_tokenizer.pad_token_id or 50256

tokenizer = TokenizerWrapper(tokenizer_hf)

print(f"✅ Tokenizer ready!")
print(f"   Vocab size: {tokenizer.vocab_size:,}")
print(f"   Max token in data: {token_ids.max()}")

if token_ids.max() >= tokenizer.vocab_size:
    print(f"   ❌ ERROR: Data has tokens beyond vocab!")
else:
    print(f"   ✅ All tokens within vocab range")

## Step 4: Create Training Batches

In [ ]:
import jax.numpy as jnp

print("="*70)
print("CREATING TRAINING BATCHES")
print("="*70)

# Training config
BATCH_SIZE = 1
SEQ_LEN = 128
TOTAL_STEPS = 50000
SAVE_EVERY = 10000  # Save every 10K steps (not 500!)
LOG_EVERY = 50

print(f"\nConfig:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Sequence length: {SEQ_LEN}")
print(f"  Total steps: {TOTAL_STEPS:,}")
print(f"  Save every: {SAVE_EVERY:,} steps")
print(f"  Checkpoints to save: {TOTAL_STEPS // SAVE_EVERY}")

# Create batches
batches = []
for i in tqdm(range(TOTAL_STEPS), desc="Creating batches"):
    offset = i * SEQ_LEN
    
    # Wrap around if we run out of data
    if offset + SEQ_LEN + 1 > len(token_ids):
        offset = offset % (len(token_ids) - SEQ_LEN - 1)
    
    input_ids = token_ids[offset:offset + SEQ_LEN]
    labels = token_ids[offset + 1:offset + SEQ_LEN + 1]
    
    batches.append({
        'input_ids': jnp.array([input_ids], dtype=jnp.int32),
        'labels': jnp.array([labels], dtype=jnp.int32)
    })

print(f"\n✅ Created {len(batches):,} batches")
print(f"   Batch shape: {batches[0]['input_ids'].shape}")

## Step 5: Load Mamba-MoE Model (509M params)

In [ ]:
import jax
from jax import random
import jax.numpy as jnp
from pathlib import Path

print("="*70)
print("LOADING MAMBA-MOE MODEL")
print("="*70)

from src.model import create_model_from_config, ModelConfig
from src.training import (
    create_train_step,
    create_train_state,
    create_learning_rate_schedule,
    create_optimizer,
    CheckpointManager,
    ConsoleLogger
)

# Verify config
import yaml
with open('configs/model_config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    
print(f"\n🔍 Config check:")
print(f"  Config vocab: {cfg['model']['vocab_size']}")
print(f"  Tokenizer vocab: {tokenizer.vocab_size}")

if cfg['model']['vocab_size'] != tokenizer.vocab_size:
    raise ValueError(f"MISMATCH! Update config to {tokenizer.vocab_size}")

print(f"  ✅ Config matches!")

# Load model
config_path = 'configs/model_config.yaml'
model = create_model_from_config(config_path)
model_config = ModelConfig.from_yaml(config_path)

print(f"\n📊 Model Architecture:")
print(f"  Name: {model_config.name}")
print(f"  Layers: {model_config.num_layers}")
print(f"  Hidden dim: {model_config.hidden_dim}")
print(f"  Mamba state dim: {model_config.mamba.state_dim}")
print(f"  MoE experts: {model_config.moe.num_experts}")
print(f"  MoE layers: {len(model_config.moe.layer_indices)}")

# Initialize parameters
print(f"\n🔄 Initializing fresh parameters...")
rng = random.PRNGKey(42)
rng, init_rng, dropout_rng = random.split(rng, 3)

dummy_input = jnp.ones((1, SEQ_LEN), dtype=jnp.int32)
variables = model.init(init_rng, dummy_input, deterministic=True)
params = variables['params']

param_count = sum(x.size for x in jax.tree_util.tree_leaves(params))
print(f"  ✓ Parameters: {param_count / 1e6:.1f}M")

# Setup training
LEARNING_RATE = 3e-4

lr_schedule = create_learning_rate_schedule(
    warmup_steps=100,
    max_learning_rate=LEARNING_RATE,
    total_steps=TOTAL_STEPS,
    schedule_type='cosine'
)

optimizer = create_optimizer(
    learning_rate_fn=lr_schedule,
    weight_decay=0.1,
    max_grad_norm=1.0
)

state = create_train_step(model, params, optimizer, lr_schedule, dropout_rng)
train_step = create_train_step(model, lr_schedule)
train_step = jax.jit(train_step)

print(f"\n✅ Model ready!")

## Step 6: Train! (50K steps, save every 10K)

**📥 Checkpoints saved every 10,000 steps** (only 5 checkpoints total!)

In [ ]:
import time

print("="*70)
print("STARTING MAMBA-MOE TRAINING")
print("="*70)

# Setup checkpoints
ckpt_dir = Path("checkpoints")
ckpt_dir.mkdir(exist_ok=True)
ckpt_manager = CheckpointManager(
    checkpoint_dir=str(ckpt_dir),
    max_to_keep=2,  # Only keep 2 latest (saves space!)
    save_interval_steps=SAVE_EVERY
)

print(f"\n⚙️ Training Config:")
print(f"  Total steps: {TOTAL_STEPS:,}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Sequence length: {SEQ_LEN}")
print(f"  Save every: {SAVE_EVERY:,} steps (only {TOTAL_STEPS // SAVE_EVERY} checkpoints!)")
print(f"  Log every: {LOG_EVERY} steps")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"\n💾 Checkpoint strategy: Keep 2 latest (saves storage)")
print(f"\n(First step compiles - takes ~1 min)")
print("="*70 + "\n")

logger = ConsoleLogger(log_interval=LOG_EVERY)
start_time = time.time()
best_loss = float('inf')

for step in range(min(TOTAL_STEPS, len(batches))):
    batch = batches[step]
    
    step_start = time.time()
    state, metrics = train_step(state, batch)
    step_time = time.time() - step_start
    
    if step == 0:
        compile_time = step_time
        print(f"✓ Compilation done ({compile_time:.1f}s)\n")
    
    metrics['step_time'] = step_time
    metrics['tokens_per_sec'] = (BATCH_SIZE * SEQ_LEN) / step_time if step > 0 else 0
    
    # Track best loss
    if float(metrics['loss']) < best_loss:
        best_loss = float(metrics['loss'])
    
    # Log
    if step % LOG_EVERY == 0 or step == 0:
        loss = float(metrics['loss'])
        ppl = float(metrics['perplexity'])
        lr = float(metrics['learning_rate'])
        tps = int(metrics['tokens_per_sec'])
        elapsed = time.time() - start_time
        eta = (elapsed / (step + 1)) * (TOTAL_STEPS - step - 1) / 3600
        
        print(f"Step {step:5d} | loss={loss:.4f} ppl={ppl:7.1f} lr={lr:.6f} | "
              f"{tps:,} tok/s | {elapsed/3600:.2f}h | ETA: {eta:.1f}h")
    
    # Save checkpoint every 10K steps
    if (step + 1) % SAVE_EVERY == 0:
        ckpt_manager.save(state, step, metadata={'loss': float(metrics['loss'])})
        print(f"\n📥 CHECKPOINT {(step+1)//SAVE_EVERY} saved at step {step+1:,}")
        print(f"   Current loss: {float(metrics['loss']):.4f} | Best: {best_loss:.4f}")
        print(f"   Storage: ~{(step+1)//SAVE_EVERY * 2}GB used (keeping 2 latest)\n")

total_time = time.time() - start_time

print(f"\n{'='*70}")
print("✅ TRAINING COMPLETE!")
print(f"{'='*70}")
print(f"\n📊 Results:")
print(f"  ✓ Trained {TOTAL_STEPS:,} steps on Mamba-MoE 509M")
print(f"  ✓ Total time: {total_time/3600:.2f} hours")
print(f"  ✓ Final loss: {float(metrics['loss']):.4f}")
print(f"  ✓ Best loss: {best_loss:.4f}")
print(f"  ✓ Final perplexity: {float(metrics['perplexity']):.1f}")
print(f"  ✓ Avg tokens/sec: {TOTAL_STEPS * BATCH_SIZE * SEQ_LEN / total_time:.0f}")
print(f"\n💾 Checkpoints:")
print(f"  Saved {TOTAL_STEPS // SAVE_EVERY} checkpoints total")
print(f"  Location: checkpoints/")
print(f"  Keeping: 2 latest (automatic cleanup)")
print(f"\n🎉 Mamba-MoE training complete!")